# Build model V0.0

# Net Data
    
| batch=1 | subdivisions=1 | Training |
| :---: | :---: | :---: |
| decay=0.0005 | angle=0 | saturation = 1.5 |
| batch=64 | subdivisions=16 | width=608 |
| height=608 | channels=3 | momentum=0.9 |
|exposure = 1.5 | hue=.1 | learning_rate=0.001|
|burn_in=1000| max_batches = 500200| policy=steps|  
|steps=400000,450000|scales=.1,.1||
 

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Dense, Conv2D, LeakyReLU, ZeroPadding2D , Add , UpSampling2D , concatenate
from keras.layers import concatenate , MaxPool2D
from keras.activations import linear
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adagrad
from keras.utils import plot_model
from keras.engine.topology import Layer
from Yolo_layer import YoloLayer

Using TensorFlow backend.


# Global Variables

In [2]:
labels = ['person'       , 'bicycle'      , 'car'          , 'motorcycle',
          'airplane'     , 'bus'          , 'train'        , 'truck', 
          'boat'         , 'traffic light', 'fire hydrant' ,'stop sign',
          'parking meter', 'bench'        , 'bird'         , 'cat', 
          'dog'          , 'horse'        , 'sheep'        ,'cow' , 
          'elephant'     , 'bear'         , 'zebra'        ,'giraffe',
          'backpack'     , 'umbrella'     , 'handbag'      ,'tie',
          'suitcase'     , 'frisbee'      , 'skis'         ,'snowboard',
          'sports ball'  , 'kite'         , 'baseball bat' , 'baseball glove',
          'skateboard'   , 'surfboard'    , 'tennis racket','bottle',
          'wine glass'   , 'cup'          , 'fork'         ,'knife',
          'spoon'        , 'bowl'         , 'banana'       ,'apple',
          'sandwich'     , 'orange'       , 'broccoli'     ,'carrot',
          'hot dog'      , 'pizza'        , 'donut'        ,'cake',
          'chair'        , 'couch'        , 'potted plant' ,'bed',
          'dining table' , 'toilet'       , 'tv'           ,'laptop', 
          'mouse'        , 'remote'       , 'keyboard'     ,'cell phone',
          'microwave'    , 'oven'         , 'toaster'      ,'sink', 
          'refrigerator' , 'book'         , 'clock'        , 'vase',
          'scissors'     , 'teddy bear'   , 'hair drier'   , 'toothbrush']

IMAGE_H = 416
IMAGE_W = 416
grid_scales = [7,17,37]
nb_class         = len(labels)

anchors          = [10,14,  23,27,  37,58,  81,82,  135,169,  344,319]

noobj_scale      = 1.0
obj_scale        = 5.0
COORD_SCALE      = 1.0
class_scale      = 1.0

batch_size       = 16
warmup_batches   = 0
TRUE_BOX_BUFFER  = 50

Epsilon          = 0.001

ALPHA = 0.1
max_grid = [480,480]
max_box_per_image = 20

# num=9
jitter=.3

truth_thresh = 1
random=1

xywh_scale = 1
ignore_thresh = .7

weights_path = 'yolov3-tiny.weights' 
count = 0

# Load weight file.

In [3]:
print('Loading weights.')
fp = open(weights_path, 'rb')
major, minor, revision = np.ndarray(shape=(3, ), dtype='int32', buffer=fp.read(12))
if (major*10+minor)>=2 and major<1000 and minor<1000:
    seen = np.ndarray(shape=(1,), dtype='int64', buffer=fp.read(8))
else:
    seen = np.ndarray(shape=(1,), dtype='int32', buffer=fp.read(4))
print('Weights Header: ', major, minor, revision, seen)

Loading weights.
Weights Header:  0 2 0 [32013312]


In [4]:
def weight_reader(size,filters,prev_layer):
    count=0
    prev_layer_shape = K.int_shape(prev_layer)
    weights_shape = (size, size, prev_layer_shape[-1], filters)
    darknet_w_shape = (filters, weights_shape[2], size, size)
    weights_size = np.product(weights_shape)
    #
    conv_bias = np.ndarray(
        shape=(filters, ),
        dtype='float32',
        buffer=fp.read(filters * 4))
    count += filters

    #
    bn_weights = np.ndarray(
        shape=(3, filters),
        dtype='float32',
        buffer=fp.read(filters * 12))
    count += 3 * filters

    bn_weight_list = [
        bn_weights[0],  # scale gamma
        conv_bias,  # shift beta
        bn_weights[1],  # running mean
        bn_weights[2]  # running var
    ]
    #
    conv_weights = np.ndarray(
        shape=darknet_w_shape,
        dtype='float32',
        buffer=fp.read(weights_size * 4))
    count += weights_size
    print(count)
    return conv_bias,bn_weights,conv_weights ,count

# Inputs

In [5]:
# net_h, net_w, 3
input_image = Input(shape=(IMAGE_H, IMAGE_W, 3),name='Main_Input') 
true_boxes  = Input(shape=(1, 1, 1, max_box_per_image, 4))
# grid_h, grid_w, nb_anchor, pc+bx+by+bh+bw+nb_class
true_yolo_1 = Input(shape=(None, None, len(anchors)//6, 5+nb_class)) 
true_yolo_2 = Input(shape=(None, None, len(anchors)//6, 5+nb_class)) 

In [6]:
# Conv
# Layer 1

size = 3
filters = 16
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,input_image)
lay1_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_1_CONV')(input_image)
lay1_2 = BatchNormalization(epsilon=Epsilon,name='Layer_1_Batch')(lay1_1)
lay1_3 = LeakyReLU(alpha=ALPHA,name='Layer_1_LRELU')(lay1_2)

# MaxPool
# Layer 2

lay2_1 = MaxPool2D(pool_size=(2,2), strides=(2,2),padding='same')(lay1_3)

# Conv
# Layer 3

size = 3
filters = 32
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay2_1)
lay3_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_3_CONV')(lay2_1)
lay3_2 = BatchNormalization(epsilon=Epsilon,name='Layer_3_Batch')(lay3_1)
lay3_3 = LeakyReLU(alpha=ALPHA,name='Layer_3_LRELU')(lay3_2)


# MaxPool
# Layer 4

lay4_1 = MaxPool2D(pool_size=(2,2), strides=(2,2),padding='same')(lay3_3)

# Conv
# Layer 5

size = 3
filters = 64
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay4_1)
lay5_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_5_CONV')(lay4_1)
lay5_2 = BatchNormalization(epsilon=Epsilon,name='Layer_5_Batch')(lay5_1)
lay5_3 = LeakyReLU(alpha=ALPHA,name='Layer_5_LRELU')(lay5_2)


# MaxPool
# Layer 6

lay6_1 = MaxPool2D(pool_size=(2,2), strides=(2,2),padding='same')(lay5_3)

# Conv
# Layer 7

size = 3
filters = 128
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay6_1)
lay7_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_7_CONV')(lay6_1)
lay7_2 = BatchNormalization(epsilon=Epsilon,name='Layer_7_Batch')(lay7_1)
lay7_3 = LeakyReLU(alpha=ALPHA,name='Layer_7_LRELU')(lay7_2)


# MaxPool
# Layer 8

lay8_1 = MaxPool2D(pool_size=(2,2), strides=(2,2),padding='same')(lay7_3)

# Conv
# Layer 9

size = 3
filters = 256
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay8_1)
lay9_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_9_CONV')(lay8_1)
lay9_2 = BatchNormalization(epsilon=Epsilon,name='Layer_9_Batch')(lay9_1)
lay9_3 = LeakyReLU(alpha=ALPHA,name='Layer_9_LRELU')(lay9_2)


# MaxPool
# Layer 10

lay10_1 = MaxPool2D(pool_size=(2,2), strides=(2,2),padding='same')(lay9_3)

# Conv
# Layer 11

size = 3
filters = 512
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay10_1)
lay11_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_11_CONV')(lay10_1)
lay11_2 = BatchNormalization(epsilon=Epsilon,name='Layer_11_Batch')(lay11_1)
lay11_3 = LeakyReLU(alpha=ALPHA,name='Layer_11_LRELU')(lay11_2)


# MaxPool
# Layer 12

lay12_1 = MaxPool2D(pool_size=(2,2), strides=(1,1),padding='same')(lay11_3)

# Conv
# Layer 13

size = 3
filters = 1024
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay12_1)
lay13_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_13_CONV')(lay12_1)
lay13_2 = BatchNormalization(epsilon=Epsilon,name='Layer_13_Batch')(lay13_1)
lay13_3 = LeakyReLU(alpha=ALPHA,name='Layer_13_LRELU')(lay13_2)


# Conv
# Layer 14

size = 1
filters = 256
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay13_3)
lay14_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_14_CONV')(lay13_3)
lay14_2 = BatchNormalization(epsilon=Epsilon,name='Layer_14_Batch')(lay14_1)
lay14_3 = LeakyReLU(alpha=ALPHA,name='Layer_14_LRELU')(lay14_2)


# Conv
# Layer 15

size = 3
filters = 512
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay14_3)
lay15_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_15_CONV')(lay14_3)
lay15_2 = BatchNormalization(epsilon=Epsilon,name='Layer_15_Batch')(lay15_1)
lay15_3 = LeakyReLU(alpha=ALPHA,name='Layer_15_LRELU')(lay15_2)

                                                      
# Conv
# Layer 16

size = 1
filters = 255
lay16 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=True, padding='same',name='Layer_16_CONV')(lay15_3)
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay15_3)


# Yolo Layer
# Layer 17

########################

# Route Layer
# Layer 18
lay18 = lay14_3

# Conv
# Layer 19

size = 1
filters = 128
lay19_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_19_CONV')(lay18)
lay19_2 = BatchNormalization(epsilon=Epsilon,name='Layer_19_Batch')(lay19_1)
lay19_3 = LeakyReLU(alpha=ALPHA,name='Layer_19_LRELU')(lay19_2)
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay18)

# Upsampling
# Layer 20

lay20 = UpSampling2D(size=(2,2))(lay19_3)

# Conv
# Layer 21

lay21 = concatenate([lay20,lay9_3],axis=-1)

# Conv
# Layer 22

size = 3
filters = 256
lay22_1 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=False, padding='same',name='Layer_22_CONV')(lay21)
lay22_2 = BatchNormalization(epsilon=Epsilon,name='Layer_22_Batch')(lay22_1)
lay22_3 = LeakyReLU(alpha=ALPHA,name='Layer_22_LRELU')(lay22_2)
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay21)

# Conv
# Layer 23

size = 1
filters = 255
lay23 = Conv2D(filters, (size,size), strides=(1, 1),use_bias=True, padding='same',name='Layer_23_CONV')(lay22_3)
#bias_buffer,bn_buffer,weight_buffer ,count_temp=weight_reader(size,filters,lay22)

# Yolo Layer
# Layer 24

In [7]:
infer_model = Model(input_image, [lay16, lay23])
print(infer_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Main_Input (InputLayer)         (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
Layer_1_CONV (Conv2D)           (None, 416, 416, 16) 432         Main_Input[0][0]                 
__________________________________________________________________________________________________
Layer_1_Batch (BatchNormalizati (None, 416, 416, 16) 64          Layer_1_CONV[0][0]               
__________________________________________________________________________________________________
Layer_1_LRELU (LeakyReLU)       (None, 416, 416, 16) 0           Layer_1_Batch[0][0]              
__________________________________________________________________________________________________
max_poolin

In [8]:
model_yaml=infer_model.to_yaml()
with open("tiny-v3.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)